### Version simple (compter le nombre de mots de l'offre dans le CV)

In [10]:
import re
from docx import Document
from nltk.corpus import stopwords
import nltk

# Téléchargement des stopwords français si nécessaire
nltk.download('stopwords')
stop_fr = set(stopwords.words('french'))

def lire_docx(path):
    """Extrait le texte brut d'un fichier Word (.docx)."""
    doc = Document(path)
    full_text = [p.text for p in doc.paragraphs]
    return "\n".join(full_text)

# --- Nettoyage de texte ---
def nettoyer_texte(texte: str) -> str:
    """Met en minuscules et supprime caractères spéciaux."""
    texte = texte.lower()
    texte = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ0-9\s]", " ", texte)
    texte = re.sub(r"\s+", " ", texte)
    return texte.strip()

# --- Extraction des mots clés d'une offre (stop words exclus) ---
def extraire_mots_cles_offre(offre_text: str):
    """
    Retourne la liste des mots clés uniques de l'offre après nettoyage
    et suppression des stop words français.
    """
    texte = nettoyer_texte(offre_text)
    mots = texte.split()
    # Exclusion des stop words
    mots_cles = [mot for mot in mots if mot not in stop_fr]
    return list(set(mots_cles))  # mots uniques

# --- Score de correspondance CV vs offre ---
def score_cv_offre(cv_text: str, offre_text: str):
    """
    Calcule le pourcentage de mots-clés de l'offre présents dans le CV.
    """
    cv_clean = nettoyer_texte(cv_text)
    cv_mots = set(cv_clean.split())

    mots_cles = extraire_mots_cles_offre(offre_text)
    # print("mots_cles:", mots_cles)

    nb_trouves = sum(1 for mot in mots_cles if mot in cv_mots)
    score = nb_trouves / len(mots_cles) if mots_cles else 0

    return score * 100  # score en pourcentage

offre = """
Dans le cadre de sa mission d’exploitation et de valorisation des données médicales, la DIDM fait face à un besoin croissant de données fiables. C’est pourquoi un nouveau poste est créé.
Vous viendrez compléter une équipe composée d’une Chargée d’études et développements à 50 % et d’un Responsable Etudes et Développements. Sous la responsabilité de ce dernier, vos missions seront les suivantes :
Construire des pipelines de données pour alimenter la BI et l’analytique.
Modéliser et structurer les flux, tables et schémas
Garantir la qualité, la fiabilité et la sécurité des données
Développer de nouveaux datasets pour la BI de la DIDM
Mettre en place des standards de développement et de bonnes pratiques
Assurer le support et la résolution des incidents sur votre périmètre...

Votre boîte à outils
Excellente maîtrise de SQL (Oracle) et solide expérience en R
Connaissances en Julia, Java ou Scala appréciées
Pratique des outils de versioning (Git, Bitbucket, Github)
Expérience avec un outil ETL, idéalement Talend
Une première approche de la dataviz (Tableau, QlikView) est un atout
"""

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eupho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Test sur un CV individuel
# Lis un CV .docx
cv_text = lire_docx("test.docx")

score = score_cv_offre(cv_text, offre)
print(f"Score de correspondance : {score:.1f} %")

mots_cles: ['scala', 'pratiques', 'bitbucket', 'sql', 'garantir', 'bi', 'viendrez', 'modéliser', 'etudes', 'mission', 'didm', 'versioning', 'qlikview', 'études', 'poste', 'julia', 'dataviz', 'assurer', 'pourquoi', 'expérience', 'médicales', 'nouveau', 'valorisation', 'datasets', 'développement', 'support', 'r', 'appréciées', 'etl', 'sécurité', 'sous', 'connaissances', 'structurer', 'outil', 'maîtrise', 'périmètre', 'approche', 'résolution', 'compléter', 'cadre', 'pratique', 'suivantes', 'responsable', 'outils', 'qualité', 'nouveaux', 'bonnes', 'place', 'première', 'fiabilité', 'boîte', 'composée', 'croissant', 'pipelines', 'atout', 'face', 'mettre', 'tables', 'développer', 'fait', 'excellente', 'incidents', 'fiables', 'talend', 'besoin', 'responsabilité', 'standards', 'idéalement', 'schémas', 'oracle', 'missions', 'équipe', 'git', 'développements', 'java', 'créé', 'tableau', 'exploitation', 'alimenter', 'chargée', 'données', 'github', 'flux', 'dernier', 'analytique', 'construire', '50'

In [11]:
# Test sur plusieurs CV 
# Faire une boucle sur les CV dans le dossier CVs
import os
from docx import Document
dossier_cvs = "CVs"
for nom_fichier in os.listdir(dossier_cvs):
    # Lis un CV .docx
    cv_text = lire_docx(f"./{dossier_cvs}/{nom_fichier}")

    score = score_cv_offre(cv_text, offre)
    print(f"Score de correspondance pour {nom_fichier} : {score:.1f} %")

Score de correspondance pour CV - Laurent D._14_01_2024.docx : 42.0 %
Score de correspondance pour CV_AnaA_20250226.docx : 31.8 %
Score de correspondance pour CV_CSA_NRJBI_20251016.docx : 28.4 %
Score de correspondance pour NRJBI_CEC_CV_Senior.docx : 17.0 %
Score de correspondance pour NRJBI_CV_EMO_202510_revisionElise.docx : 23.9 %
Score de correspondance pour test.docx : 46.6 %


### Version tout le CV offre transformés en vecteurs

In [23]:
# --- REQUIREMENTS ---
# pip install scikit-learn nltk
# ---------------------------------------------

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk, os, re
from docx import Document

# Téléchargement du stopword français
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_fr = stopwords.words('french')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eupho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# --- Fonctions ---

def lire_docx(path):
    """Extrait le texte brut d'un fichier Word (.docx)."""
    doc = Document(path)
    full_text = [p.text for p in doc.paragraphs]
    return "\n".join(full_text)

def nettoyer_texte(texte: str) -> str:
    """Nettoie et normalise un texte français."""
    texte = texte.lower()
    texte = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ0-9\s]", " ", texte)
    texte = re.sub(r"\s+", " ", texte)
    texte = re.sub(r"\d+", " ", texte)  # supprime tous les nombres
    return texte.strip()

def score_cv_contre_offre(cv_text, offre_text):
    """Calcule le score TF-IDF entre un CV unique et l'offre."""
    # Nettoyage des textes
    cv_clean = nettoyer_texte(cv_text)
    offre_clean = nettoyer_texte(offre_text)

    # TF-IDF
    vectorizer = CountVectorizer(stop_words=stop_fr, ngram_range=(1,1))
    count_words = vectorizer.fit_transform([offre_clean, cv_clean])

    # Vecteurs
    offre_vec = count_words[0:1]
    cv_vec = count_words[1:2]

    # Similarité cosinus
    score = cosine_similarity(offre_vec, cv_vec)[0][0]
    return score

In [49]:
# Lis l'offre
offre_text = """Dans le cadre de sa mission d’exploitation et de valorisation des données médicales, 
la DIDM fait face à un besoin croissant de données fiables. Vous viendrez compléter une équipe...
Excellente maîtrise de SQL (Oracle) et solide expérience en R...
Pratique des outils de versioning (Git, Bitbucket, Github)
Expérience avec un outil ETL, idéalement Talend
Une première approche de la dataviz (Tableau, QlikView) est un atout."""

# Lis un CV .docx
cv_text = lire_docx("test.docx")

# Calcul du score
score = score_cv_contre_offre(cv_text, offre_text)
print(f"Score de correspondance : {score*100:.1f} %")

Score de correspondance : 38.9 %


In [47]:
cv_text

'Ingénieur Data / Développeur BI – Spécialisation données médicales\nEmail : jean.dupont@example.com\nTéléphone : 06 45 89 22 77\n\nProfil\nData engineer expérimenté dans l’exploitation, la fiabilité et la valorisation de données médicales.\nHabitué à modéliser des flux de données, construire des pipelines ETL complexes sous Talend et SQL (Oracle), et à assurer la qualité, la sécurité et la traçabilité des données pour des environnements analytiques et décisionnels (BI, tableaux de bord, reporting).\n\nCompétences clés\nLangages : SQL (Oracle, PostgreSQL), R, Java, Scala, Julia (bases)\nOutils ETL / Data Pipeline : Talend, Airflow, DataStage\nBusiness Intelligence : Tableau, QlikView, Power BI\nVersioning : Git, Bitbucket, GitHub\nData Modeling : conception de schémas, modélisation de tables, intégration de données hétérogènes\nQualité & Sécurité : validation des datasets, contrôles de cohérence, gestion des accès et anonymisation\nBonnes pratiques : documentation, revue de code, stand

In [18]:
scores

array([0.07637796, 0.27880457])

In [36]:
#Scoring CV hors fonction
"""Calcule le score TF-IDF entre un CV unique et l'offre."""
# Nettoyage des textes
cv_clean = nettoyer_texte(cv_text)
offre_clean = nettoyer_texte(offre_text)

# TF-IDF
vectorizer = CountVectorizer(stop_words=stop_fr, ngram_range=(1,1))
count_words = vectorizer.fit_transform([offre_clean, cv_clean])

# Vecteurs
offre_vec = count_words[0:1]
cv_vec = count_words[1:2]

# Similarité cosinus
score = cosine_similarity(offre_vec, cv_vec)[0][0]
print("score:", score)

score: 0.3672651915581337


In [37]:
print(vectorizer.get_feature_names_out())

['accès' 'advanced' 'airflow' 'alimenter' 'analyste' 'analytique'
 'analytiques' 'anglais' 'anonymisation' 'approche' 'assurer' 'atout'
 'aujourd' 'autres' 'avancé' 'bases' 'bash' 'besoin' 'bi' 'bitbucket'
 'bonnes' 'bord' 'business' 'cadre' 'certification' 'clés' 'cnil' 'code'
 'cohérence' 'com' 'complexes' 'compléter' 'compétences' 'conception'
 'construction' 'construire' 'contraintes' 'contrôle' 'contrôles'
 'croissant' 'data' 'datasets' 'datastage' 'dataviz' 'didm'
 'documentation' 'données' 'dupont' 'décisionnels' 'département'
 'développement' 'développeur' 'email' 'engineer' 'engineering' 'entrepôt'
 'environnements' 'etl' 'example' 'excel' 'excellente' 'expert'
 'exploitation' 'expérience' 'expérimenté' 'face' 'fait' 'fiabilité'
 'fiables' 'flux' 'formation' 'foundations' 'france' 'français' 'garantir'
 'gestion' 'git' 'github' 'habitué' 'hui' 'hétérogènes' 'hôpital'
 'idéalement' 'incidents' 'information' 'informatique' 'ingénieur'
 'integration' 'intelligence' 'interactifs' 

In [28]:
print(offre_clean)

dans le cadre de sa mission d exploitation et de valorisation des données médicales la didm fait face à un besoin croissant de données fiables vous viendrez compléter une équipe excellente maîtrise de sql oracle et solide expérience en r pratique des outils de versioning git bitbucket github expérience avec un outil etl idéalement talend une première approche de la dataviz tableau qlikview est un atout


In [16]:
offre_vec

<1x1164 sparse matrix of type '<class 'numpy.float64'>'
	with 181 stored elements in Compressed Sparse Row format>

In [ ]:
count_words

<3x1164 sparse matrix of type '<class 'numpy.float64'>'
	with 1285 stored elements in Compressed Sparse Row format>

In [14]:
cleaned

['dans le cadre de sa mission d exploitation et de valorisation des données médicales la didm fait face à un besoin croissant de données fiables c est pourquoi un nouveau poste est créé vous viendrez compléter une équipe composée d une chargée d études et développements à 50 et d un responsable etudes et développements sous la responsabilité de ce dernier vos missions seront les suivantes construire des pipelines de données pour alimenter la bi et l analytique modéliser et structurer les flux tables et schémas garantir la qualité la fiabilité et la sécurité des données développer de nouveaux datasets pour la bi de la didm mettre en place des standards de développement et de bonnes pratiques assurer le support et la résolution des incidents sur votre périmètre votre boîte à outils excellente maîtrise de sql oracle et solide expérience en r connaissances en julia java ou scala appréciées pratique des outils de versioning git bitbucket github expérience avec un outil etl idéalement talend

In [12]:
print(len(cv_texts))

2


In [11]:
print(len(documents))

3


In [10]:
documents

['Dans le cadre de sa mission d’exploitation et de valorisation des données médicales, la DIDM fait face à un besoin croissant de données fiables. C’est pourquoi un nouveau poste est créé.\nVous viendrez compléter une équipe composée d’une Chargée d’études et développements à 50 % et d’un Responsable Etudes et Développements. Sous la responsabilité de ce dernier, vos missions seront les suivantes :\nConstruire des pipelines de données pour alimenter la BI et l’analytique.\nModéliser et structurer les flux, tables et schémas\nGarantir la qualité, la fiabilité et la sécurité des données\nDévelopper de nouveaux datasets pour la BI de la DIDM\nMettre en place des standards de développement et de bonnes pratiques\nAssurer le support et la résolution des incidents sur votre périmètre...\n\nVotre boîte à outils\nExcellente maîtrise de SQL (Oracle) et solide expérience en R\nConnaissances en Julia, Java ou Scala appréciées\nPratique des outils de versioning (Git, Bitbucket, Github)\nExpérience